<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/Pytorch011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

简单AE

In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = autoencoder()
if torch.cuda.is_available():
    model.cuda()
#model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        #img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
          #.format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.2000
epoch [2/100], loss:0.1708
epoch [3/100], loss:0.1570
epoch [4/100], loss:0.1553
epoch [5/100], loss:0.1438
epoch [6/100], loss:0.1443
epoch [7/100], loss:0.1428
epoch [8/100], loss:0.1448
epoch [9/100], loss:0.1509
epoch [10/100], loss:0.1307
epoch [11/100], loss:0.1440
epoch [12/100], loss:0.1327
epoch [13/100], loss:0.1421
epoch [14/100], loss:0.1371
epoch [15/100], loss:0.1303
epoch [16/100], loss:0.1331
epoch [17/100], loss:0.1341
epoch [18/100], loss:0.1335
epoch [19/100], loss:0.1334
epoch [20/100], loss:0.1399
epoch [21/100], loss:0.1351
epoch [22/100], loss:0.1344
epoch [23/100], loss:0.1346
epoch [24/100], loss:0.1252
epoch [25/100], loss:0.1236
epoch [26/100], loss:0.1322
epoch [27/100], loss:0.1326
epoch [28/100], loss:0.1272
epoch [29/100], loss:0.1288
epoch [30/100], loss:0.1307
epoch [31/100], loss:0.1277
epoch [32/100], loss:0.1268
epoch [33/100], loss:0.1101
epoch [34/100], loss:0.1305
epoch [35/100], loss:0.1268
epoch [36/100], loss:0.1197
e